# Crypto Pump and Dump Detection using C-LSTM and Anomaly Transformer

## Part - 1: Initial Configurations

First, lets create appropriate folders to store/load models/data.

In [1]:
import os

folder_names = ['/content/data', '/content/models']

for folder in folder_names:

    os.makedirs(folder, exist_ok=True)

    if os.path.exists(folder):
        print(f"The folder '{folder}' has been created successfully.")

    else:
        print(f"Folder already exists! '{folder}'.")

The folder '/content/data' has been created successfully.
The folder '/content/models' has been created successfully.


Lets import the necessary packages

In [3]:
import sys
import json
import time
import torch
import torch.nn.functional as F
import numpy as np
import random
import argparse

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import KFold
from data.data import create_loader, create_loaders, get_data
from models.conv_lstm import ConvLSTM
from models.anomaly_transformer import AnomalyTransformer
from models.utils import count_parameters

Lets define the path where the three datasets are present.

In [4]:
data_path1 = "/content/data/features_5S.csv.gz"
data_path2 = "/content/data/features_15S.csv.gz"
data_path3 = "/content/data/features_25S.csv.gz"

Lets define the device, we'll use for training.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Now, lets define a hyperparameter configuration class for initializing different hyperparameters.

In [27]:
class Config:

    def __init__(self, model, dataset, hyperparams):

      self.model = model
      self.dataset = dataset

      self.embedding_size = hyperparams['embedding_size']
      self.n_layers = hyperparams['n_layers']
      self.n_epochs =  hyperparams['n_epochs']
      self.kernel_size =  hyperparams['kernel_size']
      self.dropout =  hyperparams['dropout']

      self.feature_size =  hyperparams['feature_size']
      self.lambda_ =  hyperparams['lambda_']

      self.lr =  hyperparams['lr']
      self.weight_decay =  hyperparams['weight_decay']
      self.batch_size =  hyperparams['batch_size']
      self.train_ratio =  hyperparams['train_ratio']
      self.undersample_ratio =  hyperparams['undersample_ratio']
      self.segment_length =  hyperparams['segment_length']

      self.prthreshold =  hyperparams['prthreshold']
      self.kfolds =  hyperparams['kfolds']

      self.save =  hyperparams['save']
      self.validate_every_n =  hyperparams['validate_every_n']
      self.train_output_every_n =  hyperparams['train_output_every_n']
      self.time_epochs =  hyperparams['time_epochs']
      self.final_run =  hyperparams['final_run']
      self.verbose =  hyperparams['verbose']
      self.seed =  hyperparams['seed']
      self.run_count =  hyperparams['run_count']
      self.n_feats =  hyperparams['n_feats']

Now, lets define the function to create the C-LSTM model.

In [7]:
def create_conv_model(config):

    return ConvLSTM(config.n_feats, config.kernel_size, config.embedding_size,
                    config.n_layers, dropout=config.dropout).to(device)

Now, lets define the function to create the Anomaly Transformer model.

In [8]:
def create_transformer(config):

    if config.model == "AnomalyTransformer":

        return AnomalyTransformer(config.segment_length,
                                  config.feature_size, config.n_layers,
                                  config.lambda_, device).to(device)

Now, lets define the train function for one epoch.

In [9]:
def train(model, dataloader, opt, criterion, device, feature_count = 13):

    epoch_loss = 0

    for batch in dataloader:

        opt.zero_grad()
        x = batch[:, :, :feature_count].to(device)
        y = batch[:, :, -1].to(device)

        preds = model(x)
        loss = criterion(preds, y)

        loss.backward()
        opt.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

Now, lets define a function to find out the best precision-recall threshold.

In [10]:
def pick_threshold(model, dataloader, undersample_ratio, device, verbose=True,
                   feature_count=13):

    all_ys = []
    all_preds = []

    for batch in dataloader:
        with torch.no_grad():

            x = batch[:, :, :feature_count].to(device)
            y = batch[:, -1, -1].to(device)

            preds = model(x)[:, -1]
            y, preds = y.cpu().flatten(), preds.cpu().flatten()

            all_ys.append(y)
            all_preds.append(preds)

    y = torch.cat(all_ys, dim=0).cpu()
    preds = torch.cat(all_preds, dim=0).cpu()
    y = y.numpy()
    preds = preds.numpy()

    _, _, thresholds = precision_recall_curve(y, preds)

    best_f1 = 0
    best_threshold = 0

    for threshold in thresholds:

        true_pos = np.sum(preds[y == 1] >= threshold)
        false_pos = np.sum(preds[y == 0] >= threshold)
        false_neg = np.sum(preds[y == 1] < threshold)
        true_neg = np.sum(preds[y == 0] < threshold)

        false_pos /= undersample_ratio
        true_neg /= undersample_ratio

        precision = true_pos / (true_pos + false_pos)
        recall = true_pos / (true_pos + false_neg)
        f1 = 2 * precision * recall / (precision + recall)

        if f1 > best_f1:

            best_f1 = f1
            best_threshold = threshold

    if verbose:
        print(f'Best threshold: {best_threshold} (train f1: {best_f1})')

    return best_threshold

Now, lets define the validation function.

In [11]:
def validate(model, dataloader, device, verbose=True, pr_threshold=0.7,
             criterion=None, feature_count=13):

    preds_1 = []
    preds_0 = []
    all_ys = []
    all_preds = []

    epoch_loss = 0

    for batch in dataloader:
        with torch.no_grad():

            x = batch[:, :, :feature_count].to(device)
            y = batch[:, -1, -1].to(device)

            preds = model(x)[:, -1]
            y, preds = y.cpu().flatten(), preds.cpu().flatten()

            if verbose:

                preds_0.extend(preds[y == 0])
                preds_1.extend(preds[y == 1])

            all_ys.append(y)
            all_preds.append(preds)

            if criterion is not None:

                loss = criterion(preds, y)
                epoch_loss += loss.item()

    if verbose:
        print(f'Mean output at 0: {(sum(preds_0) / len(preds_0)).item():0.5f} at 1: {(sum(preds_1) / len(preds_1)).item():0.5f}')

    y = torch.cat(all_ys, dim=0).cpu()
    preds = torch.cat(all_preds, dim=0).cpu()
    preds = preds >= pr_threshold

    acc = accuracy_score(y, preds)
    precision = precision_score(y, preds, zero_division=0)
    recall = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)

    if criterion is not None:
        return acc, precision, recall, f1, epoch_loss/len(dataloader)

    else:
        return acc, precision, recall, f1

Now, lets define a method to train the model and collect metrics for all the epochs.

In [26]:
def collect_metrics_n_epochs(model, *, train_loader, test_loader, optimizer,
                             criterion, device, config, feature_count=13):

    best_metrics = np.array([0.0]*4)

    for epoch in range(config.n_epochs):

        start = time.time()
        loss = train(model, train_loader, optimizer, criterion, device,
                     feature_count)

        if (epoch + 1) % config.train_output_every_n == 0:
            print(f'Epoch {epoch + 1}{f" ({(time.time()-start):0.2f}s)" if config.time_epochs else ""} -- Train Loss: {loss:0.5f}')

        if (epoch + 1) % config.validate_every_n == 0 or config.final_run:

            if config.prthreshold > 0:
                prthreshold = config.prthreshold

            else:

                prthreshold = pick_threshold(model, train_loader,
                                             config.undersample_ratio, device,
                                             verbose=config.verbose,
                                             feature_count=feature_count)

            acc, precision, recall, f1 = validate(model, test_loader, device,
                                                  verbose=config.verbose,
                                                  pr_threshold=prthreshold,
                                                  feature_count=feature_count)

            if f1 > best_metrics[-1]:
                best_metrics = [acc, precision, recall, f1]

            print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

    return best_metrics

## Part - 2: C-LSTM (5 second Dataset)

Now, lets see how well we do on the 5 second Dataset with our C-LSTM model.

First, lets define the required hyperparameters.

In [28]:
hyperparams = {'embedding_size' : 350, 'n_layers' : 1, 'n_epochs' : 200,
               'kernel_size' : 3, 'dropout' : 0.0, 'feature_size' : 13,
               'lambda_' : 0.0001, 'lr' : 1e-3, 'weight_decay' : 0.0,
               'batch_size' : 1200, 'train_ratio' : 0.8,
               'undersample_ratio' : 0.05, 'segment_length' : 15,
               'prthreshold' : 0.5, 'kfolds' : 1, 'save' : True,
               'validate_every_n' : 10, 'train_output_every_n' : 5,
               'time_epochs' : True, 'final_run' : True, 'verbose' : False,
               'seed' : 42, 'run_count' : 1, 'n_feats' : 0}

Lets set those hyperparamters in the Config class.

In [29]:
config = Config("CLSTM", data_path1, hyperparams)

# reproducability

torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
g = torch.Generator()
g.manual_seed(config.seed)
os.environ['UJJWALS_WORKSPACE_CONFIG'] = ':4096:2'

Lets get the data now.

In [30]:
data = get_data(config.dataset, batch_size=config.batch_size,
                train_ratio=config.train_ratio,
                undersample_ratio=config.undersample_ratio,
                segment_length=config.segment_length, save=config.save)

# -1 since last column is the target value
config.n_feats = data.shape[-1] - 1 if config.feature_size == -1 else config.feature_size

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (821307, 16)
Pump 48 has 5 rows, skipping
Pump 149 has 5 rows, skipping
Pump 156 has 4 rows, skipping
Pump 180 has 1 rows, skipping
Pump 224 has 1 rows, skipping
Pump 264 has 2 rows, skipping
Pump 300 has 1 rows, skipping
Pump 327 has 1 rows, skipping
Pump 330 has 7 rows, skipping
Pump 331 has 5 rows, skipping
Skipped 32 rows total
317 pumps
821275 rows of data after processing


Lets create the model and set the loss function now.

In [31]:
criterion = torch.nn.BCELoss().to(device)

if config.model == "CLSTM":
    models = [create_conv_model] * config.run_count

else:

    criterion = torch.nn.MSELoss().to(device)
    models = [create_transformer] * config.run_count

for model_index, model_creator in enumerate(models):

    if len(models) > 1:
        print(f'Running model {model_index + 1} of {len(models)}')

Now, lets start the training process.

In [19]:
fold_metrics = np.array([0.0]*4)
sample_model = model_creator(config)

print(f'Model {type(sample_model)} using {count_parameters(sample_model)} parameters:')

model = model_creator(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr,
                              weight_decay=config.weight_decay)

if config.model == "AnomalyTransformer":
    criterion = model.loss_fn

train_loader, test_loader = create_loaders(data,
                                            train_ratio=config.train_ratio,
                                            batch_size=config.batch_size,
                                undersample_ratio=config.undersample_ratio)

best_metrics = collect_metrics_n_epochs(model, train_loader=train_loader,
                                        test_loader=test_loader,
                                        optimizer=optimizer,
                                        criterion=criterion, device=device,
                                        config=config,
                                        feature_count=config.n_feats)

fold_metrics += np.array(best_metrics)

print(f'Best F1 this run: {best_metrics[-1]}')
print()

acc, precision, recall, f1 = fold_metrics / config.kfolds

print(f'Final metrics for model {type(sample_model)} ({config.kfolds} folds)')
print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

Model <class 'models.conv_lstm.ConvLSTM'> using 997851 parameters:
Train data shape: (657020, 15, 14)
Train data shape after undersampling: (36215, 15, 14)
Test data shape: (164255, 15, 14)
60.0 segments in test data ending in anomaly
Val   -- Acc: 0.99963 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99963 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99963 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99970 -- Precision: 0.91667 -- Recall: 0.18333 -- F1: 0.30556
Epoch 5 (0.73s) -- Train Loss: 0.02007
Val   -- Acc: 0.99970 -- Precision: 0.78947 -- Recall: 0.25000 -- F1: 0.37975
Val   -- Acc: 0.99973 -- Precision: 0.75862 -- Recall: 0.36667 -- F1: 0.49438
Val   -- Acc: 0.99973 -- Precision: 0.70270 -- Recall: 0.43333 -- F1: 0.53608
Val   -- Acc: 0.99975 -- Precision: 0.72093 -- Recall: 0.51667 -- F1: 0.60194
Val   -- Acc: 0.99975 -- Precision: 0.71111 -- Recall: 0.53333 -- F1: 0.60952
Epoch 10 (0.57s) -- Trai

## Part - 3: C-LSTM (15 second Dataset)

Now, lets repeat the same process on 15 second Dataset.

In [28]:
hyperparams = {'embedding_size' : 350, 'n_layers' : 1, 'n_epochs' : 200,
               'kernel_size' : 3, 'dropout' : 0.0, 'feature_size' : 13,
               'lambda_' : 0.0001, 'lr' : 1e-3, 'weight_decay' : 0.0,
               'batch_size' : 600, 'train_ratio' : 0.8,
               'undersample_ratio' : 0.1, 'segment_length' : 15,
               'prthreshold' : 0.4, 'kfolds' : 1, 'save' : True,
               'validate_every_n' : 10, 'train_output_every_n' : 5,
               'time_epochs' : True, 'final_run' : True, 'verbose' : False,
               'seed' : 42, 'run_count' : 1, 'n_feats' : 0}

In [29]:
config = Config("CLSTM", data_path2, hyperparams)

# reproducability

torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
g = torch.Generator()
g.manual_seed(config.seed)
os.environ['UJJWALS_WORKSPACE_CONFIG'] = ':4096:2'

In [30]:
data = get_data(config.dataset, batch_size=config.batch_size,
                train_ratio=config.train_ratio,
                undersample_ratio=config.undersample_ratio,
                segment_length=config.segment_length, save=config.save)

# -1 since last column is the target value
config.n_feats = data.shape[-1] - 1 if config.feature_size == -1 else config.feature_size

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (584104, 16)
Pump 48 has 3 rows, skipping
Pump 149 has 3 rows, skipping
Pump 156 has 3 rows, skipping
Pump 180 has 1 rows, skipping
Pump 224 has 1 rows, skipping
Pump 264 has 2 rows, skipping
Pump 299 has 1 rows, skipping
Pump 300 has 1 rows, skipping
Pump 330 has 4 rows, skipping
Pump 331 has 4 rows, skipping
Skipped 23 rows total
317 pumps
584081 rows of data after processing


In [31]:
criterion = torch.nn.BCELoss().to(device)

if config.model == "CLSTM":
    models = [create_conv_model] * config.run_count

else:

    criterion = torch.nn.MSELoss().to(device)
    models = [create_transformer] * config.run_count

for model_index, model_creator in enumerate(models):

    if len(models) > 1:
        print(f'Running model {model_index + 1} of {len(models)}')

In [32]:
fold_metrics = np.array([0.0]*4)
sample_model = model_creator(config)

print(f'Model {type(sample_model)} using {count_parameters(sample_model)} parameters:')

model = model_creator(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr,
                              weight_decay=config.weight_decay)

if config.model == "AnomalyTransformer":
    criterion = model.loss_fn

train_loader, test_loader = create_loaders(data,
                                            train_ratio=config.train_ratio,
                                            batch_size=config.batch_size,
                                undersample_ratio=config.undersample_ratio)

best_metrics = collect_metrics_n_epochs(model, train_loader=train_loader,
                                        test_loader=test_loader,
                                        optimizer=optimizer,
                                        criterion=criterion, device=device,
                                        config=config,
                                        feature_count=config.n_feats)

fold_metrics += np.array(best_metrics)

print(f'Best F1 this run: {best_metrics[-1]}')
print()

acc, precision, recall, f1 = fold_metrics / config.kfolds

print(f'Final metrics for model {type(sample_model)} ({config.kfolds} folds)')
print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

Model <class 'models.conv_lstm.ConvLSTM'> using 997851 parameters:
Train data shape: (467264, 15, 14)
Train data shape after undersampling: (50339, 15, 14)
Test data shape: (116817, 15, 14)
61.0 segments in test data ending in anomaly
Val   -- Acc: 0.99948 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99969 -- Precision: 0.96296 -- Recall: 0.42623 -- F1: 0.59091
Val   -- Acc: 0.99977 -- Precision: 0.86957 -- Recall: 0.65574 -- F1: 0.74766
Val   -- Acc: 0.99978 -- Precision: 0.83019 -- Recall: 0.72131 -- F1: 0.77193
Epoch 5 (1.00s) -- Train Loss: 0.00425
Val   -- Acc: 0.99977 -- Precision: 0.77419 -- Recall: 0.78689 -- F1: 0.78049
Val   -- Acc: 0.99976 -- Precision: 0.78947 -- Recall: 0.73770 -- F1: 0.76271
Val   -- Acc: 0.99979 -- Precision: 0.77273 -- Recall: 0.83607 -- F1: 0.80315
Val   -- Acc: 0.99983 -- Precision: 0.82540 -- Recall: 0.85246 -- F1: 0.83871
Val   -- Acc: 0.99984 -- Precision: 0.85000 -- Recall: 0.83607 -- F1: 0.84298
Epoch 10 (0.98s) -- Trai

## Part - 4: C-LSTM (25 second Dataset)

Now, lets repeat the same process on the 25 second Dataset.

In [33]:
hyperparams = {'embedding_size' : 350, 'n_layers' : 1, 'n_epochs' : 200,
               'kernel_size' : 3, 'dropout' : 0.0, 'feature_size' : 13,
               'lambda_' : 0.0001, 'lr' : 1e-3, 'weight_decay' : 0.0,
               'batch_size' : 600, 'train_ratio' : 0.8,
               'undersample_ratio' : 0.05, 'segment_length' : 15,
               'prthreshold' : 0.65, 'kfolds' : 1, 'save' : True,
               'validate_every_n' : 10, 'train_output_every_n' : 5,
               'time_epochs' : True, 'final_run' : True, 'verbose' : False,
               'seed' : 42, 'run_count' : 1, 'n_feats' : 0}

In [38]:
config = Config("CLSTM", data_path3, hyperparams)

# reproducability

torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
g = torch.Generator()
g.manual_seed(config.seed)
os.environ['UJJWALS_WORKSPACE_CONFIG'] = ':4096:2'

In [35]:
data = get_data(config.dataset, batch_size=config.batch_size,
                train_ratio=config.train_ratio,
                undersample_ratio=config.undersample_ratio,
                segment_length=config.segment_length, save=config.save)

# -1 since last column is the target value
config.n_feats = data.shape[-1] - 1 if config.feature_size == -1 else config.feature_size

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (482157, 16)
Pump 48 has 3 rows, skipping
Pump 149 has 2 rows, skipping
Pump 156 has 3 rows, skipping
Pump 180 has 1 rows, skipping
Pump 224 has 1 rows, skipping
Pump 264 has 2 rows, skipping
Pump 299 has 1 rows, skipping
Pump 300 has 1 rows, skipping
Pump 330 has 3 rows, skipping
Pump 331 has 3 rows, skipping
Skipped 20 rows total
317 pumps
482137 rows of data after processing


In [36]:
criterion = torch.nn.BCELoss().to(device)

if config.model == "CLSTM":
    models = [create_conv_model] * config.run_count

else:

    criterion = torch.nn.MSELoss().to(device)
    models = [create_transformer] * config.run_count

for model_index, model_creator in enumerate(models):

    if len(models) > 1:
        print(f'Running model {model_index + 1} of {len(models)}')

In [38]:
fold_metrics = np.array([0.0]*4)
sample_model = model_creator(config)

print(f'Model {type(sample_model)} using {count_parameters(sample_model)} parameters:')

model = model_creator(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr,
                              weight_decay=config.weight_decay)

if config.model == "AnomalyTransformer":
    criterion = model.loss_fn

train_loader, test_loader = create_loaders(data,
                                            train_ratio=config.train_ratio,
                                            batch_size=config.batch_size,
                                undersample_ratio=config.undersample_ratio)

best_metrics = collect_metrics_n_epochs(model, train_loader=train_loader,
                                        test_loader=test_loader,
                                        optimizer=optimizer,
                                        criterion=criterion, device=device,
                                        config=config,
                                        feature_count=config.n_feats)

fold_metrics += np.array(best_metrics)

print(f'Best F1 this run: {best_metrics[-1]}')
print()

acc, precision, recall, f1 = fold_metrics / config.kfolds

print(f'Final metrics for model {type(sample_model)} ({config.kfolds} folds)')
print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

Model <class 'models.conv_lstm.ConvLSTM'> using 997851 parameters:
Train data shape: (385709, 15, 14)
Train data shape after undersampling: (22722, 15, 14)
Test data shape: (96428, 15, 14)
62.0 segments in test data ending in anomaly
Val   -- Acc: 0.99936 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99936 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99936 -- Precision: 0.00000 -- Recall: 0.00000 -- F1: 0.00000
Val   -- Acc: 0.99969 -- Precision: 1.00000 -- Recall: 0.51613 -- F1: 0.68085
Epoch 5 (0.44s) -- Train Loss: 0.01312
Val   -- Acc: 0.99975 -- Precision: 1.00000 -- Recall: 0.61290 -- F1: 0.76000
Val   -- Acc: 0.99977 -- Precision: 0.97619 -- Recall: 0.66129 -- F1: 0.78846
Val   -- Acc: 0.99977 -- Precision: 0.91667 -- Recall: 0.70968 -- F1: 0.80000
Val   -- Acc: 0.99981 -- Precision: 0.94000 -- Recall: 0.75806 -- F1: 0.83929
Val   -- Acc: 0.99981 -- Precision: 0.94000 -- Recall: 0.75806 -- F1: 0.83929
Epoch 10 (0.45s) -- Train

## Part - 5: Anomaly Transformer (5 second Dataset)

Now, lets see how are Anomaly Transformer performs on the 5 second Dataset.

The process is exactly the same. Just the hyperparameters we set are going to change.

In [33]:
hyperparams = {'embedding_size' : 350, 'n_layers' : 4, 'n_epochs' : 50,
               'kernel_size' : 3, 'dropout' : 0.0, 'feature_size' : 13,
               'lambda_' : 0.0001, 'lr' : 1e-3, 'weight_decay' : 0.0,
               'batch_size' : 32, 'train_ratio' : 0.8,
               'undersample_ratio' : 0.05, 'segment_length' : 15,
               'prthreshold' : 0.48, 'kfolds' : 1, 'save' : True,
               'validate_every_n' : 10, 'train_output_every_n' : 5,
               'time_epochs' : True, 'final_run' : True, 'verbose' : False,
               'seed' : 42, 'run_count' : 1, 'n_feats' : 0}

In [34]:
config = Config("AnomalyTransformer", data_path1, hyperparams)

# reproducability

torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
g = torch.Generator()
g.manual_seed(config.seed)
os.environ['CUBLAS_'] = ':4096:2'

In [35]:
data = get_data(config.dataset, batch_size=config.batch_size,
                train_ratio=config.train_ratio,
                undersample_ratio=config.undersample_ratio,
                segment_length=config.segment_length, save=config.save)

# -1 since last column is the target value
config.n_feats = data.shape[-1] - 1 if config.feature_size == -1 else config.feature_size

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (821307, 16)
Pump 48 has 5 rows, skipping
Pump 149 has 5 rows, skipping
Pump 156 has 4 rows, skipping
Pump 180 has 1 rows, skipping
Pump 224 has 1 rows, skipping
Pump 264 has 2 rows, skipping
Pump 300 has 1 rows, skipping
Pump 327 has 1 rows, skipping
Pump 330 has 7 rows, skipping
Pump 331 has 5 rows, skipping
Skipped 32 rows total
317 pumps
821275 rows of data after processing


In [36]:
criterion = torch.nn.BCELoss().to(device)

if config.model == "CLSTM":
    models = [create_conv_model] * config.run_count

else:

    criterion = torch.nn.MSELoss().to(device)
    models = [create_transformer] * config.run_count

for model_index, model_creator in enumerate(models):

    if len(models) > 1:
        print(f'Running model {model_index + 1} of {len(models)}')

In [43]:
fold_metrics = np.array([0.0]*4)
sample_model = model_creator(config)

print(f'Model {type(sample_model)} using {count_parameters(sample_model)} parameters:')

model = model_creator(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr,
                              weight_decay=config.weight_decay)

if config.model == "AnomalyTransformer":
    criterion = model.loss_fn

train_loader, test_loader = create_loaders(data,
                                            train_ratio=config.train_ratio,
                                            batch_size=config.batch_size,
                                undersample_ratio=config.undersample_ratio)

best_metrics = collect_metrics_n_epochs(model, train_loader=train_loader,
                                        test_loader=test_loader,
                                        optimizer=optimizer,
                                        criterion=criterion, device=device,
                                        config=config,
                                        feature_count=config.n_feats)

fold_metrics += np.array(best_metrics)

print(f'Best F1 this run: {best_metrics[-1]}')
print()

acc, precision, recall, f1 = fold_metrics / config.kfolds

print(f'Final metrics for model {type(sample_model)} ({config.kfolds} folds)')
print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

Model <class 'models.anomaly_transformer.AnomalyTransformer'> using 3030 parameters:
Train data shape: (657020, 15, 14)
Train data shape after undersampling: (36215, 15, 14)
Test data shape: (164255, 15, 14)
60.0 segments in test data ending in anomaly
Val   -- Acc: 0.99966 -- Precision: 1.00000 -- Recall: 0.06667 -- F1: 0.12500
Val   -- Acc: 0.99967 -- Precision: 0.87500 -- Recall: 0.11667 -- F1: 0.20588
Val   -- Acc: 0.99969 -- Precision: 0.90909 -- Recall: 0.16667 -- F1: 0.28169
Val   -- Acc: 0.99970 -- Precision: 0.86667 -- Recall: 0.21667 -- F1: 0.34667
Epoch 5 (87.11s) -- Train Loss: 0.00077
Val   -- Acc: 0.99973 -- Precision: 0.86364 -- Recall: 0.31667 -- F1: 0.46341
Val   -- Acc: 0.99977 -- Precision: 0.84848 -- Recall: 0.46667 -- F1: 0.60215
Val   -- Acc: 0.99979 -- Precision: 0.83784 -- Recall: 0.51667 -- F1: 0.63918
Val   -- Acc: 0.99980 -- Precision: 0.75472 -- Recall: 0.66667 -- F1: 0.70796
Val   -- Acc: 0.99979 -- Precision: 0.71186 -- Recall: 0.70000 -- F1: 0.70588
Epoch

## Part - 6: Anomaly Transformer (15 second Dataset)

In [40]:
config = Config("AnomalyTransformer", data_path2, hyperparams)

# reproducability

torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
g = torch.Generator()
g.manual_seed(config.seed)
os.environ['UJJWALS_WORKSPACE_CONFIG'] = ':4096:2'

In [41]:
data = get_data(config.dataset, batch_size=config.batch_size,
                train_ratio=config.train_ratio,
                undersample_ratio=config.undersample_ratio,
                segment_length=config.segment_length, save=config.save)

# -1 since last column is the target value
config.n_feats = data.shape[-1] - 1 if config.feature_size == -1 else config.feature_size

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (584104, 16)
Pump 48 has 3 rows, skipping
Pump 149 has 3 rows, skipping
Pump 156 has 3 rows, skipping
Pump 180 has 1 rows, skipping
Pump 224 has 1 rows, skipping
Pump 264 has 2 rows, skipping
Pump 299 has 1 rows, skipping
Pump 300 has 1 rows, skipping
Pump 330 has 4 rows, skipping
Pump 331 has 4 rows, skipping
Skipped 23 rows total
317 pumps
584081 rows of data after processing


In [42]:
criterion = torch.nn.BCELoss().to(device)

if config.model == "CLSTM":
    models = [create_conv_model] * config.run_count

else:

    criterion = torch.nn.MSELoss().to(device)
    models = [create_transformer] * config.run_count

for model_index, model_creator in enumerate(models):

    if len(models) > 1:
        print(f'Running model {model_index + 1} of {len(models)}')

In [43]:
fold_metrics = np.array([0.0]*4)
sample_model = model_creator(config)

print(f'Model {type(sample_model)} using {count_parameters(sample_model)} parameters:')

model = model_creator(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr,
                              weight_decay=config.weight_decay)

if config.model == "AnomalyTransformer":
    criterion = model.loss_fn

train_loader, test_loader = create_loaders(data,
                                            train_ratio=config.train_ratio,
                                            batch_size=config.batch_size,
                                undersample_ratio=config.undersample_ratio)

best_metrics = collect_metrics_n_epochs(model, train_loader=train_loader,
                                        test_loader=test_loader,
                                        optimizer=optimizer,
                                        criterion=criterion, device=device,
                                        config=config,
                                        feature_count=config.n_feats)

fold_metrics += np.array(best_metrics)

print(f'Best F1 this run: {best_metrics[-1]}')
print()

acc, precision, recall, f1 = fold_metrics / config.kfolds

print(f'Final metrics for model {type(sample_model)} ({config.kfolds} folds)')
print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

Model <class 'models.anomaly_transformer.AnomalyTransformer'> using 3030 parameters:
Train data shape: (467264, 15, 14)
Train data shape after undersampling: (26837, 15, 14)
Test data shape: (116817, 15, 14)
61.0 segments in test data ending in anomaly
Val   -- Acc: 0.99953 -- Precision: 1.00000 -- Recall: 0.09836 -- F1: 0.17910
Val   -- Acc: 0.99961 -- Precision: 1.00000 -- Recall: 0.24590 -- F1: 0.39474
Val   -- Acc: 0.99963 -- Precision: 0.95000 -- Recall: 0.31148 -- F1: 0.46914
Val   -- Acc: 0.99967 -- Precision: 0.95833 -- Recall: 0.37705 -- F1: 0.54118
Epoch 5 (63.54s) -- Train Loss: 0.00089
Val   -- Acc: 0.99972 -- Precision: 0.96667 -- Recall: 0.47541 -- F1: 0.63736
Val   -- Acc: 0.99964 -- Precision: 0.91304 -- Recall: 0.34426 -- F1: 0.50000
Val   -- Acc: 0.99967 -- Precision: 0.92308 -- Recall: 0.39344 -- F1: 0.55172
Val   -- Acc: 0.99961 -- Precision: 0.94444 -- Recall: 0.27869 -- F1: 0.43038
Val   -- Acc: 0.99963 -- Precision: 0.95000 -- Recall: 0.31148 -- F1: 0.46914
Epoch

## Part - 7: Anomaly Transformer (25 second Dataset)

In [44]:
config = Config("AnomalyTransformer", data_path3, hyperparams)

# reproducability

torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
g = torch.Generator()
g.manual_seed(config.seed)
os.environ['UJJWALS_WORKSPACE_CONFIG'] = ':4096:2'

In [45]:
data = get_data(config.dataset, batch_size=config.batch_size,
                train_ratio=config.train_ratio,
                undersample_ratio=config.undersample_ratio,
                segment_length=config.segment_length, save=config.save)

# -1 since last column is the target value
config.n_feats = data.shape[-1] - 1 if config.feature_size == -1 else config.feature_size

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (482157, 16)
Pump 48 has 3 rows, skipping
Pump 149 has 2 rows, skipping
Pump 156 has 3 rows, skipping
Pump 180 has 1 rows, skipping
Pump 224 has 1 rows, skipping
Pump 264 has 2 rows, skipping
Pump 299 has 1 rows, skipping
Pump 300 has 1 rows, skipping
Pump 330 has 3 rows, skipping
Pump 331 has 3 rows, skipping
Skipped 20 rows total
317 pumps
482137 rows of data after processing


In [46]:
criterion = torch.nn.BCELoss().to(device)

if config.model == "CLSTM":
    models = [create_conv_model] * config.run_count

else:

    criterion = torch.nn.MSELoss().to(device)
    models = [create_transformer] * config.run_count

for model_index, model_creator in enumerate(models):

    if len(models) > 1:
        print(f'Running model {model_index + 1} of {len(models)}')

In [47]:
fold_metrics = np.array([0.0]*4)
sample_model = model_creator(config)

print(f'Model {type(sample_model)} using {count_parameters(sample_model)} parameters:')

model = model_creator(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr,
                              weight_decay=config.weight_decay)

if config.model == "AnomalyTransformer":
    criterion = model.loss_fn

train_loader, test_loader = create_loaders(data,
                                            train_ratio=config.train_ratio,
                                            batch_size=config.batch_size,
                                undersample_ratio=config.undersample_ratio)

best_metrics = collect_metrics_n_epochs(model, train_loader=train_loader,
                                        test_loader=test_loader,
                                        optimizer=optimizer,
                                        criterion=criterion, device=device,
                                        config=config,
                                        feature_count=config.n_feats)

fold_metrics += np.array(best_metrics)

print(f'Best F1 this run: {best_metrics[-1]}')
print()

acc, precision, recall, f1 = fold_metrics / config.kfolds

print(f'Final metrics for model {type(sample_model)} ({config.kfolds} folds)')
print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')

Model <class 'models.anomaly_transformer.AnomalyTransformer'> using 3030 parameters:
Train data shape: (385709, 15, 14)
Train data shape after undersampling: (22743, 15, 14)
Test data shape: (96428, 15, 14)
62.0 segments in test data ending in anomaly
Val   -- Acc: 0.99938 -- Precision: 1.00000 -- Recall: 0.03226 -- F1: 0.06250
Val   -- Acc: 0.99942 -- Precision: 1.00000 -- Recall: 0.09677 -- F1: 0.17647
Val   -- Acc: 0.99942 -- Precision: 1.00000 -- Recall: 0.09677 -- F1: 0.17647
Val   -- Acc: 0.99941 -- Precision: 1.00000 -- Recall: 0.08065 -- F1: 0.14925
Epoch 5 (54.42s) -- Train Loss: 0.00056
Val   -- Acc: 0.99942 -- Precision: 1.00000 -- Recall: 0.09677 -- F1: 0.17647
Val   -- Acc: 0.99948 -- Precision: 1.00000 -- Recall: 0.19355 -- F1: 0.32432
Val   -- Acc: 0.99957 -- Precision: 1.00000 -- Recall: 0.33871 -- F1: 0.50602
Val   -- Acc: 0.99964 -- Precision: 1.00000 -- Recall: 0.43548 -- F1: 0.60674
Val   -- Acc: 0.99961 -- Precision: 1.00000 -- Recall: 0.38710 -- F1: 0.55814
Epoch 